# **Maximum Flow Problem**

The Maximum Flow Problem is a fundamental problem in network optimization where the objective is to find the maximum amount of flow that can be pushed from a source node to a sink node in a network. Each edge in the network has a capacity that restricts the maximum flow through that edge. The problem has wide applications, including transportation systems, telecommunications, and flow control in manufacturing processes.

**_MIP Model_**

**_Indices and Parameters_**

-   $i,j$ = Indices representing nodes in the graph.
-   $(i,j)$ = Directed arc from node $i$ to node $j$.

-   $c_{ij}$: Capacity of arc $(i,j)$, indicating the maximum flow that can traverse this arc.
-   $s$: Index of the source node, where the flow originates.
-   $t$: Index of the sink node, where the flow is targeted.

**_Decision Variables_**

-   $f_{i,j}$: flow on arc $(i,j)$.This represents the flow amount that is sent from node $i$ to node $j$


### Objective Function

Maximize the net flow from the source node \(s\) to the sink node \(t\):

$$
\text{Maximize} \quad \sum_{j | (s,j) \in A} f_{sj} - \sum_{i | (i,s) \in A} f_{is}
$$

### Constraints

1. **Capacity Constraints**:

   Ensure that the flow on each arc does not exceed its capacity:

   $$
   f_{ij} \leq c_{ij} \quad \forall (i, j) \in A
   $$

2. **Flow Conservation Constraints**:

   At every node (except the source and sink), the incoming flow must equal the outgoing flow:

   $$
   \sum_{j | (j,i) \in A} f_{ji} = \sum_{j | (i,j) \in A} f_{ij} \quad \forall i \in V \setminus \{s, t\}
   $$

3. **Non-negativity Constraints**:

   The flow on all arcs must be non-negative:

   $$
   f_{ij} \geq 0 \quad \forall (i, j) \in A
   $$



# Code

In [5]:
import pulp
class MaxFlowProblem:
    def __init__(self, nodes, arcs, s, t, c):
        self.nodes = nodes # List of all nodes
        self.arcs = arcs # List of tuples (i, j) representing directed arcs
        self.s = s
        self.t = t
        self.c = c  # Dictionary with capacities for arcs

        # Initialize the problem
        self.problem = pulp.LpProblem("MaxFlowProblem", pulp.LpMaximize)

        #Decision variables
        self.f = pulp.LpVariable.dicts("flow", self.arcs, lowBound=0, cat=pulp.LpContinuous)
    
    def build_model(self):

        #objective function
        self.problem += pulp.lpSum([self.f[i,j] for (i,j) in self.arcs if i == self.s]) -  pulp.lpSum(self.f[i,j] for (i,j) in self.arcs if j == self.s)

        #constraints
        #1:Capacity Constraints
        for i, j in self.arcs:
            self.problem += self.f[i,j] <= self.c[(i,j)]
        
        #2:Flow Conservation Constraints
        for node in self.nodes:
            if node != self.s and node != self.t:
                incoming_flow = pulp.lpSum([self.f[i, node] for i, _ in self.arcs if _ == node])
                outgoing_flow = pulp.lpSum([self.f[node, j] for _, j in self.arcs if _ == node])
                self.problem += (incoming_flow == outgoing_flow)
        
        #3:Non-negativity Constraints
        #already is taken care of because we set the lower bound to zero

    def solve(self):
        # Optionally use CPLEX if available, otherwise default to PuLP's default solver
        try:
            self.problem.solve(pulp.CPLEX_PY(msg=True))
        except pulp.PulpSolverError:
            print("CPLEX solver not available, using default solver.")
            self.problem.solve()

        if self.problem.status == pulp.LpStatusOptimal:
            print(f"Optimal value is: {pulp.value(self.problem.objective)}")
            # Print the flows on each arc
            print("Flows on arcs:")
            for (i, j) in self.arcs:
                print(f"Flow from {i} to {j}: {self.f[i,j].varValue}")
        else:
            print("No optimal solution found.")

In [6]:
# Example of usage
nodes = ['s', 'a', 'b', 't']
arcs = [('s', 'a'), ('a', 'b'), ('b', 't'), ('a', 't')]
capacities = {('s', 'a'): 10, ('a', 'b'): 15, ('b', 't'): 10, ('a', 't'): 10}
problem = MaxFlowProblem(nodes, arcs, 's', 't', capacities)
problem.build_model()
problem.solve()

Version identifier: 22.1.1.0 | 2023-06-15 | d64d5bd77
CPXPARAM_Read_DataCheck                          1


Found incumbent of value 0.000000 after 0.00 sec. (0.00 ticks)
Tried aggregator 1 time.
MIP Presolve eliminated 6 rows and 4 columns.
All rows and columns eliminated.
Presolve time = 0.02 sec. (0.00 ticks)

Root node processing (before b&c):
  Real time             =    0.02 sec. (0.01 ticks)
Parallel b&c, 16 threads:
  Real time             =    0.00 sec. (0.00 ticks)
  Sync time (average)   =    0.00 sec.
  Wait time (average)   =    0.00 sec.
                          ------------
Total (root+branch&cut) =    0.02 sec. (0.01 ticks)
Cplex status= 101
Optimal value is: 10.0
Flows on arcs:
Flow from s to a: 10.0
Flow from a to b: 10.0
Flow from b to t: 10.0
Flow from a to t: 0.0
